In [9]:
import numpy as np
import pandas as pd

In [147]:
data=pd.read_csv('Munich House raw.csv',delimiter='\t')
data.head()

,index0,terrace,bedrooms,construction_year,date_month,date_year,date_year_month,floor,rooms,price_room,price_surface,price,surface,zip
0,0,yes,2,1986,3,2016,2016_03,up floor,3,108333.33333333333,3915.6626506024095,325000,83,85221
1,1,yes,NAN,1972,3,2016,2016_03,up floor,3,115000.0,4107.142857142857,345000,84,85521
2,2,yes,2,1968,3,2016,2016_03,up floor,3,123333.33333333333,4805.194805194805,370000,77,81369
3,3,yes,2,1963,3,2016,2016_03,up floor,3,133000.0,4244.68085106383,399000,94,82110
4,4,yes,2,1973,3,2016,2016_03,up floor,3,122933.33333333333,4789.61038961039,368800,77,81735


In [148]:
data=data.drop('index0',1)

In [149]:
data.head()

,terrace,bedrooms,construction_year,date_month,date_year,date_year_month,floor,rooms,price_room,price_surface,price,surface,zip
0,yes,2,1986,3,2016,2016_03,up floor,3,108333.33333333333,3915.6626506024095,325000,83,85221
1,yes,NAN,1972,3,2016,2016_03,up floor,3,115000.0,4107.142857142857,345000,84,85521
2,yes,2,1968,3,2016,2016_03,up floor,3,123333.33333333333,4805.194805194805,370000,77,81369
3,yes,2,1963,3,2016,2016_03,up floor,3,133000.0,4244.68085106383,399000,94,82110
4,yes,2,1973,3,2016,2016_03,up floor,3,122933.33333333333,4789.61038961039,368800,77,81735


In [150]:
data.dtypes

terrace              object
bedrooms             object
construction_year    object
date_month            int64
date_year             int64
date_year_month      object
floor                object
rooms                object
price_room           object
price_surface        object
price                object
surface              object
zip                   int64
dtype: object

In [151]:
data['terrace'].isna().sum()
# these NAN are string NAN not np.nan
# for handling nan values, first we have to convert them into np.nan

0

In [152]:
data=data.replace('NAN', np.nan)

In [153]:
data.isna().sum()
# now we have missing data to handle

terrace                0
bedrooms             319
construction_year     49
date_month             0
date_year              0
date_year_month        0
floor                113
rooms                 34
price_room            35
price_surface        212
price                  1
surface              211
zip                    0
dtype: int64

In [154]:
# columns having % missing data
(data.isna().sum()/len(data))*100

terrace               0.000000
bedrooms             36.708861
construction_year     5.638665
date_month            0.000000
date_year             0.000000
date_year_month       0.000000
floor                13.003452
rooms                 3.912543
price_room            4.027618
price_surface        24.395857
price                 0.115075
surface              24.280783
zip                   0.000000
dtype: float64

In [155]:
data['floor'].value_counts()

up floor        687
ground floor     69
Name: floor, dtype: int64